In [ ]:
# Import libraries
from selenium import webdriver
from selenium.webdriver.support.select import Select
import datetime
import os
import glob
from zipfile import ZipFile
import time
from selenium.webdriver.common.by import By

In [ ]:
# Declare constants
download_loc = './Zips'
move_loc = './Flights/'
csv_ext = 'csv'
zip_ext = 'zip'

In [ ]:
# Declare options for chrome and instantiate driver
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_loc}
chromeOptions.add_experimental_option("prefs",prefs)
wd = webdriver.Chrome(executable_path='./chromedriver', options=chromeOptions )
wd.get("https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236")

In [ ]:
# Declare input list to load from the website as constants
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
years_back = 2
year = datetime.datetime.today().year - 1
years_list = [year - i for i in range(years_back+1)]
years_list = [str(i) for i in years_list]
input_list = ['FlightDate', 'Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginState', 'DestAirportID', 'Dest', 'DestCityName', 'DestState', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CRSElapsedTime', 'ActualElapsedTime', 'Flights', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [ ]:
# Function to select values in the select box based on the value. 
def loop_select_keys(eName, eValue):
    attempt = 1
    while True:
        try:
            time.sleep(0.5)
            return Select(wd.find_element_by_id(eName)).select_by_visible_text(eValue)
            print(attempt)
        except (StaleElementReferenceException, ElementNotInteractableException, MoveTargetOutOfBoundsException, NoSuchElementException, InvalidElementStateException) :
            if attempt == 10:
                raise
            attempt += 1

In [ ]:
# Select the state at which flight data needs to be downloaded
loop_select_keys("GEOGRAPHY", "Colorado")

# Loop through the years in the listbox on the UI
for year in years_list:

    # Select Year
    loop_select_keys("XYEAR", year)
    print("Selected" + year)
    time.sleep(2)
    # Select Month
    for month in month_list:
        
        loop_select_keys("FREQUENCY", month)
        print("Selected" + month)
        time.sleep(2)
        # If the Code here is the first time then select all the inputs for download
        if(year == "2019" and month == "January"):
            for item in input_list:
                wd.find_element_by_xpath("//*[@title='" + item + "']").click()
            time.sleep(2)
            # Function to scroll to the top
            wd.execute_script("window.scrollTo(0, window.scrollY - 2500)")
        time.sleep(2)
        
        # Click button to download
        wd.find_element_by_class_name('tsbutton').click()
        # Sleep is high as my internet is slow and it takes utmost 20 secs to download data for a month
        time.sleep(20)
        # Go to the download location
        if(os.path.exists(download_loc)):
            os.chdir(download_loc)
            # Unzip the file
            for zname in glob.glob('*.{}'.format(zip_ext)):
                # opening the zip file in READ mode 
                with ZipFile(zname, 'r') as zip: 
                    # printing all the contents of the zip file 
                    zip.printdir() 
                    # extracting all the files  
                    zip.extractall()
                    for fname in glob.glob('*.{}'.format(csv_ext)):
                        # Rename the file and move it to a different location
                        os.rename(fname, move_loc + year + "_" + month + "." + csv_ext)
        for dfiles in glob.glob('*.{}'.format(zip_ext)):
            time.sleep(5)
            # Remove the zip file from the zips folder
            os.remove(dfiles)
        print("Completed processing ", year + "_" + month)